# PyUnfold Examples

In [ ]:
from pyunfold import iterative_unfold

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

## Example 1: Two-bin case

The simplest case consists of two bins

### Observed distribution

In [ ]:
data = [100, 150]
data_err = np.sqrt(data)

In [ ]:
labels = ['Class 1', 'Class 2']
sns.barplot(labels, data, palette="viridis")
plt.ylabel('Observed distribution')

### Response matrix

In [ ]:
response = [[0.9, 0.1],
            [0.1, 0.9]]
response_err = [[0.01, 0.01],
                [0.01, 0.01]]

In [ ]:
sns.heatmap(response, annot=True, cmap='viridis', xticklabels=labels, yticklabels=labels)

### Detection efficiencies

In [ ]:
efficiencies = [1.0, 1.0]
efficiencies_err = [0.01, 0.01]

### Perform iterative unfolding

In [ ]:
unfolded_results = iterative_unfold(data, data_err,
                                    response, response_err,
                                    efficiencies, efficiencies_err)

In [ ]:
unfolded_results

In [ ]:
unfolded_dist = unfolded_results['unfolded']
unfolded_dist

In [ ]:
sns.barplot(labels, unfolded_dist, palette="viridis")
plt.ylabel('Unfolded distribution')

In [ ]:
df = pd.DataFrame({'label': 2*labels,
                   'dist': list(data) + list(unfolded_dist),
                   'type': ['Observed', 'Observed', 'Unfolded', 'Unfolded']})
df

In [ ]:
sns.factorplot(x='label', y='dist', data=df,
               hue='type', kind='bar', palette='viridis',
               legend=False)
plt.xlabel('')
plt.ylabel('Distributions')
plt.legend()
plt.show()

## Example 2: Multi-bin case

### True and observed distributions

In [ ]:
num_samples = int(1e5)
true_samples = np.random.normal(loc=0.0, scale=1.0, size=num_samples)

For this example, the observed (measured) samples are the true samples with some random Gaussian noise added.

In [ ]:
observed_samples = true_samples + np.random.normal(loc=0.2, scale=0.5, size=num_samples)

In [ ]:
bins = np.linspace(-3, 3, 21)

In [ ]:
plt.hist(true_samples, bins=bins, histtype='step', lw=3,
         alpha=0.7, density=True, label='True distribution')
plt.hist(observed_samples, bins=bins, histtype='step', lw=3,
         alpha=0.7, density=True, label='Observed distribution')
plt.xlabel('X')
plt.ylabel('A.U.')
plt.legend()
plt.show()

In [ ]:
data_true, _ = np.histogram(true_samples, bins=bins)
data_true = np.array(data_true, dtype=float)
data_true

In [ ]:
data_observed, _ = np.histogram(observed_samples, bins=bins)
data_observed

Statistical Poisson errors

In [ ]:
data_observed_err = np.sqrt(data_observed)
data_observed_err

### Response matrix

In [ ]:
response, _, _ = np.histogram2d(observed_samples, true_samples, bins=bins)
response_err = np.sqrt(response)

Column normalize the response histogram to give $P(E|C)$

In [ ]:
response = response / response.sum(axis=0)
response_err = response_err / response.sum(axis=0)

In [ ]:
plt.imshow(response, origin='lower')
plt.colorbar()
plt.show()

### Efficiencies

For now, we'll assume uniform efficiencies of 1.

In [ ]:
efficiencies = response.sum(axis=0)
efficiencies

In [ ]:
efficiencies_err = np.full_like(efficiencies, 0.1, dtype=float)
efficiencies_err

### Perform iterative unfolding

In [ ]:
unfolded_results = iterative_unfold(data=data_observed,
                                    data_err=data_observed_err,
                                    response=response,
                                    response_err=response_err,
                                    efficiencies=efficiencies,
                                    efficiencies_err=efficiencies_err,
                                    ts='ks',
                                    ts_stopping=0.001)

In [ ]:
unfolded_results

Comparison of true, observed, and unfolded distributions

In [ ]:
bin_midpoints = (bins[1:] + bins[:-1]) / 2
plt.hist(true_samples, bins=bins, histtype='step', lw=3,
         alpha=0.7,
         label='True distribution')
plt.hist(observed_samples, bins=bins, histtype='step', lw=3,
         alpha=0.7,
         label='Observed distribution')

plt.errorbar(bin_midpoints, unfolded_results['unfolded'],
             yerr=unfolded_results['sys_err'],
             alpha=0.8,
             elinewidth=2,
             capsize=3,
             ls='None', marker='o', ms=5, 
             label='Unfolded distribution')

plt.xlabel('X')
plt.ylabel('Counts')
plt.legend()
plt.show()